In [1]:
import numpy as np
import math
from bounding_box_utils.bounding_box_utils import iou, convert_coordinates
from ssd_encoder_decoder.matching_utils import match_bipartite_greedy, match_multi
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [2]:
sess = tf.Session()

K.set_session(sess)

In [3]:
gt = np.load("outputs/predder.npy")

In [4]:
gt1 = gt[1]['predictions_1']
gt2 = gt[1]['predictions_2']
gt1_proj = gt[1]['predictions_1_proj_tot']
gt2_proj = gt[1]['predictions_2_proj_tot']

In [5]:
pred_1 = np.load("outputs/predictions_1_14.npy")
pred_1_proj = np.load("outputs/predictions_1_proj_14.npy")
pred_2 = np.load("outputs/predictions_2_14.npy")
pred_2_proj = np.load("outputs/predictions_2_proj_14.npy")

labels:  (5, 4)
y_encoded:  (17292, 4)
similarities:  (5, 17292)
bipartite_matches:  (5,)
similarities after bipartite:  (5, 17292)
matches:  (53,)  -  (53,)
similarities after matches:  (5, 17292)

In [6]:
pred_1 = pred_1_proj[:,:,:18]
gt_1 = gt1_proj[:,:,:18]
gt_2 = gt2_proj[:,:,18:]

In [7]:
filterer = np.where(pred_1[:,:,0]>0.5)
filterer2 = np.where(gt_1[0,:,-1]!=99)

In [8]:
# src = src[filterer]
gt_1 = gt_1[0,filterer2]
gt_2 = gt_2[0,filterer2]

In [9]:
# src = src[0,:,:]
print(pred_1.shape)
print(gt_1.shape)
print(gt_2.shape)

(1, 17292, 18)
(1, 4, 18)
(1, 4, 18)


In [10]:
gt_2[filterer2].shape

IndexError: index 4845 is out of bounds for axis 0 with size 1

In [ ]:
iou_out = iou(pred_1[:,-16:-12],gt_1[:,-16:-12], coords='centroids', mode='outer_product', border_pixels='half')

In [ ]:
bipartite_matches = match_bipartite_greedy(weight_matrix=iou_out)
bipartite_matches.shape

In [ ]:
bipartite_matches

In [ ]:
pred_2 = pred_1_proj[:,:,18:]

In [ ]:
pred_2 = pred_2[:,bipartite_matches,:]

In [ ]:
pred_2 = pred_2[0,:,:]

In [ ]:
pred_2

In [ ]:
def log_loss(y_true, y_pred):
    y_pred = tf.maximum(y_pred, 1e-15)
    # Compute the log loss
    log_loss = -tf.reduce_sum(y_true * tf.log(y_pred), axis=-1)
    return log_loss


In [ ]:
gt_2[:,:14].shape

In [ ]:
pred_2[:,:14].shape

In [ ]:
log = log_loss(gt_2[:,:14], pred_2[:,:14])

In [ ]:
K.eval(log)

In [ ]:
def smooth_L1_loss(y_true, y_pred):
    '''
    Compute smooth L1 loss, see references.

    Arguments:
        y_true (nD tensor): A TensorFlow tensor of any shape containing the ground truth data.
            In this context, the expected tensor has shape `(batch_size, #boxes, 4)` and
            contains the ground truth bounding box coordinates, where the last dimension
            contains `(xmin, xmax, ymin, ymax)`.
        y_pred (nD tensor): A TensorFlow tensor of identical structure to `y_true` containing
            the predicted data, in this context the predicted bounding box coordinates.

    Returns:
        The smooth L1 loss, a nD-1 Tensorflow tensor. In this context a 2D tensor
        of shape (batch, n_boxes_total).

    References:
        https://arxiv.org/abs/1504.08083
    '''
    absolute_loss = tf.abs(y_true - y_pred)
    square_loss = 0.5 * (y_true - y_pred)**2
    l1_loss = tf.where(tf.less(absolute_loss, 1.0), square_loss, absolute_loss - 0.5)
    return tf.reduce_sum(l1_loss, axis=-1)



In [ ]:
smooth = smooth_L1_loss(gt_2[:,:14], pred_2[:,:14])

In [ ]:
K.eval(smooth)

In [ ]:
def convert_coordinates_tf(tensor):
    d = 0
    ind = 0
    tensor1 = tensor
    tensor1[..., ind] = (tensor[..., ind] + tensor[..., ind+2]) / 2.0 # Set cx
    tensor1[..., ind+1] = (tensor[..., ind+1] + tensor[..., ind+3]) / 2.0 # Set cy
    tensor1[..., ind+2] = tensor[..., ind+2] - tensor[..., ind] + d # Set w
    tensor1[..., ind+3] = tensor[..., ind+3] - tensor[..., ind+1] + d # Set h
    return tensor1


In [ ]:
def iou_tf(boxes1, boxes2, coords='centroids', mode='outer_product', border_pixels='half'):
#     boxes1 = convert_coordinates_tf(boxes1, start_index=0, conversion='centroids2corners')
    boxes1 = tf.py_func(convert_coordinates_tf, [boxes1], tf.float64)

#     boxes2 = convert_coordinates_tf(boxes2, start_index=0, conversion='centroids2corners')
    print(K.eval(boxes1))
#     # Compute the IoU.
#     # Compute the interesection areas.
#     intersection_areas = intersection_area_(boxes1, boxes2, coords=coords, mode=mode)

#     m = boxes1.shape[0] # The number of boxes in `boxes1`
#     n = boxes2.shape[0] # The number of boxes in `boxes2`

#     # Compute the union areas.

#     # Set the correct coordinate indices for the respective formats.
#     if coords == 'corners':
#         xmin = 0
#         ymin = 1
#         xmax = 2
#         ymax = 3
#     elif coords == 'minmax':
#         xmin = 0
#         xmax = 1
#         ymin = 2
#         ymax = 3

#     if border_pixels == 'half':
#         d = 0
#     elif border_pixels == 'include':
#         d = 1 # If border pixels are supposed to belong to the bounding boxes, we have to add one pixel to any difference `xmax - xmin` or `ymax - ymin`.
#     elif border_pixels == 'exclude':
#         d = -1 # If border pixels are not supposed to belong to the bounding boxes, we have to subtract one pixel from any difference `xmax - xmin` or `ymax - ymin`.

#     if mode == 'outer_product':

#         boxes1_areas = np.tile(np.expand_dims((boxes1[:,xmax] - boxes1[:,xmin] + d) * (boxes1[:,ymax] - boxes1[:,ymin] + d), axis=1), reps=(1,n))
#         boxes2_areas = np.tile(np.expand_dims((boxes2[:,xmax] - boxes2[:,xmin] + d) * (boxes2[:,ymax] - boxes2[:,ymin] + d), axis=0), reps=(m,1))

#     elif mode == 'element-wise':

#         boxes1_areas = (boxes1[:,xmax] - boxes1[:,xmin] + d) * (boxes1[:,ymax] - boxes1[:,ymin] + d)
#         boxes2_areas = (boxes2[:,xmax] - boxes2[:,xmin] + d) * (boxes2[:,ymax] - boxes2[:,ymin] + d)

#     union_areas = boxes1_areas + boxes2_areas - intersection_areas

#     return intersection_areas / union_areas


In [ ]:
iou_out = iou_tf(tf.convert_to_tensor(gt_1[:,-16:-12]), tf.convert_to_tensor(pred_1[:,-16:-12]), coords='centroids', mode='outer_product', border_pixels='half')
